In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, mean

In [2]:
list_years = [str(i) for i in range(2010,2022)]

In [ ]:
# Create a URL through you can access the Spark UI
#get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
# Access the URL
#!curl -s http://localhost:4040/api/tunnels 

In [3]:
spark = SparkSession.builder \
    .appName("ENEM analysis") \
    .config("spark.jars", "./path/postgresql-42.7.2.jar") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [4]:
df_analysis = {}

url = "jdbc:postgresql://localhost:5433/ENEM_Data"

properties = {
    "user": "admin",
    "password": "*********",
    "driver": "org.postgresql.Driver"
}

In [5]:
# general function for queries using partition
def load_query(query, url, partitionColumn = "CO_PROVA_CN", numPartitions = "8", lowerbound = "89", upperbound = "108"):
    # Analyze min and maximum values
    df = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", query) \
    .option("numPartitions",numPartitions) \
    .option("partitionColumn", "\"" + partitionColumn + "\"") \
    .option("lowerBound", lowerbound) \
    .option("upperBound", upperbound) \
    .option("user", "admin") \
    .option("password", "*********")
    df_part = df.load()
    return df_part

In [6]:
for year in list_years:
    df_analysis[year] = load_query("(select * from \"Data_years\".\"Analise_" + year + "\")  as subq",
                          url,
                         "Faixa_Etaria",
                         "8",
                         "1",
                         "20"
                         )

In [7]:
df_all = df_analysis['2010']

In [8]:
for year in list_years:
    if (year != '2010'):
        df_all = df_all.union(df_analysis[year])

In [9]:
df_all.count()

69816478

AVG CN Score

In [10]:
df_all.select(mean(col('NU_NOTA_CN'))).show()

+------------------+
|   avg(NU_NOTA_CN)|
+------------------+
|483.33408304403133|
+------------------+



AVG CH Score

In [ ]:
df_all.select(mean(col('NU_NOTA_CH'))).show()

AVG LC Score

In [ ]:
df_all.select(mean(col('NU_NOTA_LC'))).show()

AVG MT Score

In [ ]:
df_all.select(mean(col('NU_NOTA_MT'))).show()

In [10]:
spark.stop()